In [1]:
import sys,os
import xarray as xr
import numpy as np
import scipy as spy
import pickle 

In [2]:
import matplotlib
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import cartopy.feature as cfeature
from cartopy.util import add_cyclic_point
from collections import OrderedDict 

In [3]:
import time as timestamp 
import importlib

In [4]:
sys.path.append("/home/disk/p/mkb22/Documents/si_analysis_kb/LIMs/SI_LIMs/")
import LIM_utils as lim

import LIM_utils_kb as limkb
import LIM_stats_kb as statskb
import LIM_plot_kb as plotkb
import LIM_building as limbuild

sys.path.append("/home/disk/kalman2/mkb22/pyLMR/")
import LMR_utils

In [5]:
#--------------------------------------------------
# START USER PARAMETERS
#--------------------------------------------------
# number of EOFs to retain for the LIM state vector
ntrunc = 30 # EOF truncation for individual fields (reduced-state space)
nmodes = 30 # number of coupled EOFs for LIM state space (truncated-state space)
#nmodes = 25
nmodes_sic = 50
#modes_sic = 20

mo='all'
#mo=0

# forecast lead time in months that defines the LIM training
tau = 1

# training data defined by the first ntrain times
# fraction of years used in training
# ensures that start year of both train and validation data is january
# nyearstrain = 30
# ntrain = int(nyearstrain*12)
# nvalid = int(38*12)
# nyearsvalid = 38-nyearstrain
# nvalidtimes = 1872

# variables to include in the LIM (note "vars" is a Python command)
#limvars = ['tas','zg']
#limvars = ['tas','rlut','zg']
#limvars = ['sic']
#limvars = ['tas','sic']
#limvars = ['tas','sic','zg','psl','pr','tos']
#limvars = ['tas','psl','tos','sit','sic']
# limvars = ['tas','tos','psl','sit','sic']
# limvars_nosic = ['tas','tos','psl','sit']
limvars = ['tas','tos','psl','sic']
#limvars = ['sic']
limvars_nosic = []
nvars = len(limvars)

# specify the model source 
#train_dsource = 'satellite'
train_dsource = 'era5'
#train_dsource = 'mpi_hist_kb'
#train_dsource = 'ccsm4_lm_kb'
#valid_dsource = 'satellite'
#valid_dsource = 'cmip6_mpi_hist'
#valid_dsource = 'ccsm4_lm_kb'
valid_dsource = 'cmip6_mpi_hist'
#valid_dsource = 'mpi_lm_kb'
#valid_dsource = 'ccsm4_lm_kb'

sic_separate = True
Insamp = False

date_of_interest = '202109026'

In [ ]:
if 'hist_ssp585' in train_dsource: 
    folder_add = 'hist_ssp585_concatenated/'
elif 'hist' in train_dsource: 
    folder_add = 'historical/'
elif 'lm' in train_dsource: 
    folder_add = 'last_millennium/'
elif 'era5' in train_dsource: 
    folder_add = 'era5/'

In [7]:
# Start with truncated training data: 
#mod_folder = 'truncated_model_data/last_millennium/'
mod_folder = 'truncated_model_data/'+folder_add
#mod_filename = '_ntrunc30_mpi_lm_kb_20210503.pkl'
#mod_filename = '_ntrunc30_monthall_ccsm4_hist_kb_20210507_ntrain1_standtest.pkl'
# mod_filename = '_ntrunc30_monthall_mpi_hist_kb_20210507_ntrain1_standtest.pkl'
#mod_filename = '_ntrunc'+str(ntrunc)+'_monthall_cmip6_mpi_hist_20210819_ntrain_1850_1999_standtest.pkl'
#mod_filename = '_ntrunc'+str(ntrunc)+'_monthall_mpi_hist_kb_20210824_ntrain_1850_1999_standtest.pkl'
mod_filename = ('_ntrunc'+str(ntrunc)+'_monthall_'+train_dsource+
                '_ntrain_1850_2005_'+date_of_interes+'.pkl')

#mod_filename = '_ntrunc30_mpi_lm_kb_20210507_ntrain0_3_standtest.pkl'
#mod_sic_filename = '_ntrunc50_mpi_lm_kb_20210406.pkl'
#mod_sic_filename = '_ntrunc50_monthall_ccsm4_hist_kb_20210507_ntrain1_standtest.pkl'
#mod_sic_filename = '_ntrunc50_monthall_mpi_hist_kb_20210507_ntrain1_standtest.pkl'
#mod_sic_filename = '_ntrunc'+str(nmodes_sic)+'_monthall_cmip6_mpi_hist_20210819_ntrain_1850_1999_standtest.pkl'
#mod_sic_filename = '_ntrunc'+str(nmodes_sic)+'_monthall_mpi_hist_kb_20210824_ntrain_1850_1999_standtest.pkl'
mod_sic_filename = ('_ntrunc'+str(ntrunc)+'_monthall_'+train_dsource+
                    '_ntrain_1850_2005_'+date_of_interes+'.pkl')



#mod_sic_filename = '_ntrunc50_month0_mpi_hist_kb_20210727_ntrain1_standtest.pkl'
#mod_sic_filename = '_ntrunc50_mpi_lm_kb_20210507_ntrain0_3_standtest.pkl'
mod_filename

'_ntrunc30_monthall_cmip6_mpi_hist_20210826_ntrain_1850_2005_standtest.pkl'

In [16]:
mod_filename[-23:-14]

'1850_2005'

In [26]:
exp_setup = {}
exp_setup['mo'] = mo
exp_setup['tau'] = tau
exp_setup['ntrunc'] = ntrunc 
exp_setup['nmodes_sic'] = nmodes_sic
exp_setup['limvars'] = limvars
exp_setup['train_dsource'] = train_dsource
exp_setup['valid_dsource'] = valid_dsource 
exp_setup['sic_separate'] = sic_separate
exp_setup['Insamp'] = Insamp
exp_setup['mod_folder'] = mod_folder
exp_setup['mod_filename'] = mod_filename
exp_setup['mod_sic_filename'] = mod_sic_filename
exp_setup['nyearsvalid'] = 100
#exp_setup['nyearstot'] = 156
exp_setup['nyearstot'] = 155
exp_setup['nyears_startvalid'] = 0
exp_setup['ntrain']=((exp_setup['nyearstot']*12)-(exp_setup['nyearsvalid'] *12))/(exp_setup['nyearstot']*12)

In [27]:
month_names = ['Jan','Feb','Mar','Apr','May','June','July','Aug','Sept','Oct','Nov','Dec']

In [28]:
import run_forecast_model_data as rf

In [54]:
importlib.reload(rf)
importlib.reload(limbuild)
importlib.reload(limkb)

<module 'LIM_utils_kb' from '/home/disk/p/mkb22/Documents/si_analysis_kb/LIMs/SI_LIMs/LIM_utils_kb.py'>

In [55]:
#--------------------------------------------------
### Build L from truncated data: 
#--------------------------------------------------

save_folder = '/home/disk/p/mkb22/Documents/si_analysis_kb/LIMs/SI_LIMs/LIMs_saved/'

LIMd = rf.build_L(exp_setup, save_folder, save=True)

Loading from torical/areacella_fx_MPI-ESM1-2-LR_historical_r1i1p1f1_gn.nc
Loading from orical/areacello_Ofx_MPI-ESM1-2-LR_historical_r1i1p1f1_gn.nc
Loading truncated tas
Loading truncated tos
Loading truncated sit
Loading truncated sic
working on tas
working on tos
working on sit
working on sic
0, tas
1, tos
2, sit
Training LIM with tau = 1
Number of positive eigenvalues = 0.0
saving in: /home/disk/p/mkb22/Documents/si_analysis_kb/LIMs/SI_LIMs/LIMs_saved/L_cmip6_mpi_hist_ntrain_1850_2005_tas30_tos30_sit30_sic50_20210902.pkl


In [31]:
#--------------------------------------------------
### Load pre-build L: 
#--------------------------------------------------
date_of_interest = '20210902'
L_folder = '/home/disk/p/mkb22/Documents/si_analysis_kb/LIMs/SI_LIMs/LIMs_saved/'
#L_filename = 'L_'+train_dsource+'_ntrain_1850_2004_tas30_tos30_sit30_sic50_'+date_of_interest+'.pkl'
L_filename = 'L_'+train_dsource+'_ntrain_'+mod_filename[-23:-14]+'_tas30_tos30_sit30_sic50_'+date_of_interest+'.pkl'
print('Loading... '+L_filename)

LIMd = pickle.load(open(L_folder+L_filename, 'rb'))

Loading... L_cmip6_mpi_hist_ntrain_1850_2005_tas30_tos30_sit30_sic50_20210902.pkl


In [57]:
LIMd['W_all']['tas'].shape

(18432,)

## Run Forecast: 

In [32]:
lags = [0,1,2,3,4,5,6]

if LIMd['npos_eigenvalues'] >0: 
    adj = True
else: 
    adj = False

exp_setup['lags'] = lags
exp_setup['adj'] = adj
exp_setup['remove_climo'] = True
exp_setup['detrend'] = True
exp_setup['nyr_train'] = None
exp_setup['Insamp'] = Insamp

f_folder = '/home/disk/p/mkb22/Documents/si_analysis_kb/LIMs/SI_LIMs/LIMs_saved/'

In [40]:
forecast = rf.run_forecast(LIMd,exp_setup, f_folder, verbose=True, save=False)

Validation data: cmip6_cesm2_hist
Loading from tas
Loading from rical/tas_Amon_CESM2_historical_r1i1p1f1_gn_185001-201412.nc
time dimension: 1850 - 1959
(1320,)
removing climotology...
detrending...
(55296, 1320)
(55296, 1320)
-----------------------------------------------------
completed in 7.640288829803467 seconds
-----------------------------------------------------
Validation shape: (55296, 1320)
from 1850 thru 1959


ValueError: operands could not be broadcast together with shapes (18432,1) (55296,1320) 

In [ ]:
# L_folder = '/home/disk/p/mkb22/Documents/si_analysis_kb/LIMs/SI_LIMs/LIMs_saved/'
# L_filename = 'Forecast_'+train_dsource+'_ntrain_1850_2005_tas30_tos30_sit30_sic50_'+date_of_interest+'.pkl'

# forecast = pickle.load(open(L_folder+L_filename, 'rb'))

In [43]:
fdic_valid = limkb.build_training_dic(exp_setup['valid_dsource'])

Ptrunc_valid = {}
var_dict_valid = {}
ntims = len(exp_setup['lags'])
print('Validation data: '+exp_setup['valid_dsource'])
verbose=True

for k, var in enumerate(['tas']): 
    tecut = exp_setup['nyears_startvalid']+exp_setup['nyearstot']-exp_setup['nyearsvalid']
    X_var_valid, var_dict_valid = limkb.load_data(var, var_dict_valid, fdic_valid, remove_climo=exp_setup['remove_climo'], 
                                            detrend=exp_setup['detrend'], verbose=verbose, 
                                            tscut=exp_setup['nyears_startvalid'], tecut=tecut)

Validation data: cmip6_cesm2_hist
Loading from tas
Loading from rical/tas_Amon_CESM2_historical_r1i1p1f1_gn_185001-201412.nc
time dimension: 1850 - 1959
(1320,)
removing climotology...
detrending...
(55296, 1320)
(55296, 1320)
-----------------------------------------------------
completed in 7.630411624908447 seconds
-----------------------------------------------------


In [48]:
X_var_valid.shape, LIMd['W_all']['tas'].shape

((55296, 1320), (18432,))

In [ ]:
forecast_validation = rf.validate_forecast_monthly(forecast, exp_setup['limvars'], 1, exp_setup, LIMd, f_folder, 
                                                   iplot=True, save=True)

In [ ]:
# # L_filename = ('Forecast_validation_'+train_dsource+
# #               '_ntrain_1850_2005_validation_'+valid_dsource+'_0_100_tas30_tos30_sit30_sic50_'+
# #               date_of_interest+'.pkl')

# L_filename = 'Forecast_validation_mpi_hist_kb_ntrain_1850_2005_tas30_tos30_sit30_sic50_20210826.pkl'
# print('Opening...'+ L_filename)

# forecast_validation_monthly = pickle.load(open(L_folder+L_filename, 'rb'))

In [ ]:
forecast_validation_lags = rf.validate_forecast_lagged(forecast, exp_setup['limvars'], exp_setup, LIMd, 
                                                       f_folder, iplot=True, save=True)

In [ ]:
# # L_filename = ('Forecast_validation_lagged_'+train_dsource+
# #               '_ntrain_1850_2005_validation_'+valid_dsource+'_0_100_tas30_tos30_sit30_sic50_'+
# #               date_of_interest+'.pkl')

# L_filename = 'Forecast_validation_lagged_mpi_hist_kb_ntrain_1850_2005_tas30_tos30_sit30_sic50_20210826.pkl'
# print('Opening...'+ L_filename)

# forecast_validation_lags = pickle.load(open(L_folder+L_filename, 'rb'))

## Run AR1 Forecast: 

In [ ]:
valid_vars=limvars
ar1f_folder = '/home/disk/p/mkb22/Documents/si_analysis_kb/LIMs/SI_LIMs/LIMs_saved/'

ar1cast = rf.ar1_forecast_valid_by_month(LIMd['P_train'], forecast['P_train_valid'], LIMd,
                                         exp_setup, valid_vars, month_names, ar1f_folder, forecast,  
                                         lag=None, iplot=True, save=True, save_decomp=False)

In [ ]:
# L_folder = '/home/disk/p/mkb22/Documents/si_analysis_kb/LIMs/SI_LIMs/LIMs_saved/'
# # L_filename = ('AR1_forecast_monthly_'+train_dsource+
# #               '_ntrain_1850_1999_tas30_tos30_psl30_sit30_sic50_'+date_of_interest+'.pkl')

# L_filename = ('AR1_forecast_monthly_'+train_dsource+'__ntrain_1850_2005_validation_'+
#               valid_dscource+'_'+exp_setup['nyears_startvalid']+'_'+exp_setup['nyearsvalid']+
#               '_tas30_tos30_sit30_sic50_'+date_of_interest+'.pkl')
# print('Opening...'+L_filename)

# ar1cast = pickle.load(open(L_folder+L_filename, 'rb'))

In [ ]:
ar1f_folder = '/home/disk/p/mkb22/Documents/si_analysis_kb/LIMs/SI_LIMs/LIMs_saved/'

ar1cast_lags = rf.ar1_forecast_valid_by_lag(LIMd['P_train'], forecast['P_train_valid'], LIMd, exp_setup, 
                                            exp_setup['limvars'], month_names, ar1f_folder, forecast, 
                                            iplot=True, save=True, save_decomp=False)

In [ ]:
# L_folder = '/home/disk/p/mkb22/Documents/si_analysis_kb/LIMs/SI_LIMs/LIMs_saved/'
# # L_filename = ('AR1_forecast_lagged_'+train_dsource+
# #               '_ntrain_1850_1999_tas30_tos30_psl30_sit30_sic50_'+date_of_interest+'.pkl')

# L_filename = ('AR1_forecast_lagged_'+train_dsource+'__ntrain_1850_2005_validation_'+
#               valid_dscource+'_'+exp_setup['nyears_startvalid']+'_'+exp_setup['nyearsvalid']+
#               '_tas30_tos30_sit30_sic50_'+date_of_interest+'.pkl')
# print('Opening...'+L_filename

# ar1cast_lags = pickle.load(open(L_folder+L_filename, 'rb'))

In [ ]:
# from datetime import date

# today = date.today()

# #Year-month-day
# today_date = today.strftime("%Y%m%d")

# ar1_validation = {}

# ar1_validation['validation_stats_ar1_lags'] = ar1cast_lags['validation_stats_ar1_lags']
# ar1_validation['validation_stats_ar1'] = ar1cast['validation_stats_ar1']

# ar1f_filename = ('AR1_validation_all_'+exp_setup['valid_dsource']+'_'+
#                 str(exp_setup['nyears_startvalid'])+'_'+str(exp_setup['nyearsvalid'])+'_'+
#                 (str(exp_setup['ntrunc'])+"_").join(exp_setup['limvars'])+
#                 str(exp_setup['nmodes_sic'])+'_'+today_date+'.pkl')

# print('saving in: '+ar1f_folder+ar1f_filename)
# pickle.dump(ar1_validation, open(ar1f_folder+ar1f_filename, "wb" ) )

In [ ]:
# L_folder = '/home/disk/p/mkb22/Documents/si_analysis_kb/LIMs/SI_LIMs/LIMs_saved/'
# L_filename = ('AR1_validation_all_mpi_lm_kb_0_100_tas30_tos30_sit30_sic50_20210826.pkl')

# ar1_validation = pickle.load(open(L_folder+L_filename, 'rb'))

## Validation by lag: 

In [ ]:
if 'cmip6' in train_dsource: 
    tcmip='CMIP6'
else: 
    tcmip='CMIP5'

var = 'sic'
# start_tim = exp_setup['nyears_startvalid']
# end_tim = exp_setup['nyears_startvalid']+(exp_setup['nyearsvalid']*12)
valid_yrs_str = (str(forecast['var_dict_valid'][var]['time'][0])[:4]+'_'+
                 str(forecast['var_dict_valid'][var]['time'][-1])[:4])

if 'hist' in train_dsource:
    texp = 'Historical'
elif 'lm' in train_dsource: 
    texp = 'LM'
elif 'ssp585' in train_dsource: 
    texp = 'SSP585'
    
if 'hist' in valid_dsource:
    vexp = 'Historical'
elif 'lm' in valid_dsource: 
    vexp = 'LM'
elif 'ssp585' in valid_dsource: 
    vexp = 'SSP585'
    
if 'cesm2' in train_dsource:
    tmod = 'CESM2'
elif 'mpi' in train_dsource: 
    tmod = 'MPI'
elif 'gfdl' in train_dsource: 
    tmod = 'GFDL'
elif 'ccsm4' in train_dsource: 
    tmod = 'CCSM4'
    
if 'cesm2' in valid_dsource:
    vmod = 'CESM2'
elif 'mpi' in valid_dsource: 
    vmod = 'MPI'
elif 'gfdl' in valid_dsource: 
    vmod = 'GFDL'
elif 'ccsm4' in valid_dsource: 
    vmod = 'CCSM4'

In [ ]:
fig,axs = plt.subplots(2,3,figsize=(20,10))
#fig,axs= plt.subplots(2, 2)

ax = axs.flatten()

for i,var in enumerate(exp_setup['limvars']):
    ax[i].plot(exp_setup['lags'][1:],forecast_validation_lags['validation_stats_lags'][var]['corr_tot'][1:]**2, 
               label='Correlation',linewidth=4)
    ax[i].plot(exp_setup['lags'][1:],ar1cast_lags['validation_stats_ar1_lags'][var]['corr_tot'][:-1]**2, 
               label='AR(1) Correlation',linewidth=4, 
               linestyle='--', color='tab:blue',alpha=0.5)

    ax[i].plot(exp_setup['lags'][1:],forecast_validation_lags['validation_stats_lags'][var]['ce_tot'][1:],
               label='CE',linewidth=3.0)
    ax[i].plot(exp_setup['lags'][1:],ar1cast_lags['validation_stats_ar1_lags'][var]['ce_tot'][:-1],
               label='AR(1) CE',linewidth=3.0, 
               linestyle='--', color='tab:orange', alpha=0.5)

    ax[i].set_xticks(np.arange(0,11,1))
    ax[i].set_xticklabels(np.arange(0,11,1),fontsize=12)
    ax[i].set_ylim(0,1)
    ax[i].set_xlim(1,len(exp_setup['lags'][1:]))
    ax[i].grid(axis='both')
    ax[i].set_title(var, fontsize=14)
#    ax[i].text(0.1,0.05, ("Mean R$^2$ = "+str(np.round(np.mean(validation_stats[var]['corr_tot']**2),2))),fontsize=14)
    
if Insamp==True:     
    plt.suptitle(('LIM trained on all months \n Training years: '+str(850)+'-'+str(int(850+ntrain*1000))+
                 ', Validation years: Historical'), fontsize=16)
else: 
    plt.suptitle(('LIM trained on all months \n Training years: '+tcmip+' '+tmod+' '+texp+' '+
                  exp_setup['mod_filename'][-23:-14]+', Validation years: '+vmod+' '+vexp+' '+
                  valid_yrs_str), fontsize=16)

# ax[i].text(0.1,0.05, ("Training years: "+str(850)+'-'+str(850+ntrain*1000)),fontsize=14)
# ax[i].text(0.1,0.05, ("Validation years: "+str(850+ntrain*1000)+'-1850'),fontsize=14)


ax[1].legend(loc='lower left', fontsize=14)
ax[2].set_xlabel('Lag (months)', fontsize=14)
ax[1].set_xlabel('Lag (months)', fontsize=14)
ax[3].set_xlabel('Lag (months)', fontsize=14)

plt.subplots_adjust(top=0.9)

In [ ]:
fig,axs = plt.subplots(2,3,figsize=(20,10))
#fig,axs= plt.subplots(2, 2)

ax = axs.flatten()

for i,var in enumerate(exp_setup['limvars']):
    ax[i].plot(exp_setup['lags'][1:],forecast_validation_lags['validation_stats_lags'][var]['gm_var_ratio'][1:], 
               label='LIM',linewidth=3.0, color='purple')
    ax[i].plot(exp_setup['lags'][1:],ar1cast_lags['validation_stats_ar1_lags'][var]['gm_var_ratio'][:-1], 
               label='AR(1)',linewidth=3.0, linestyle='--', color='purple',alpha=0.5)

    ax[i].set_xticks(np.arange(0,11,1))
    ax[i].set_xticklabels(np.arange(0,11,1),fontsize=12)
    ax[i].set_ylim(0,1.1)
    ax[i].set_xlim(1,len(exp_setup['lags'][1:]))
    ax[i].grid(axis='both')
    ax[i].set_title(var, fontsize=14)
    
    ax[i].axhline(1.0, color='k', linewidth=2, label='No skill')
    
if Insamp==True:     
    plt.suptitle(('LIM trained on all months \n Training years: '+str(850)+'-'+str(int(850+ntrain*1000))+
                 ', Validation years: Historical'), fontsize=16)
else:     
    plt.suptitle(('LIM trained on all months \n Training years: '+tcmip+' '+tmod+' '+texp+' '+
                  exp_setup['mod_filename'][-23:-14]+', Validation years: '+vmod+' '+vexp+' '+
                  valid_yrs_str), fontsize=16)

# ax[i].text(0.1,0.05, ("Training years: "+str(850)+'-'+str(850+ntrain*1000)),fontsize=14)
# ax[i].text(0.1,0.05, ("Validation years: "+str(850+ntrain*1000)+'-1850'),fontsize=14)


ax[1].legend(loc='lower right', fontsize=14)
ax[2].set_xlabel('Lag (months)', fontsize=14)
ax[1].set_xlabel('Lag (months)', fontsize=14)
ax[3].set_xlabel('Lag (months)', fontsize=14)

plt.subplots_adjust(top=0.9)

## Validation by month: 

In [ ]:
fig,axs = plt.subplots(2,3,figsize=(20,9))
#fig,axs= plt.subplots(2, 2)

ax = axs.flatten()

for i,var in enumerate(exp_setup['limvars']):
    ax[i].plot(forecast_validation['validation_stats'][var]['corr_tot']**2, label='Correlation',linewidth=3.0)
    ax[i].plot(ar1cast['validation_stats_ar1'][var]['corr_tot']**2, label='AR(1) Correlation',linewidth=3.0, 
               linestyle='--', color='tab:blue',alpha=0.5)

    ax[i].plot(forecast_validation['validation_stats'][var]['ce_tot'],label='CE',linewidth=3.0)
    ax[i].plot(ar1cast['validation_stats_ar1'][var]['ce_tot'],label='AR(1) CE',linewidth=3.0, 
               linestyle='--', color='tab:orange', alpha=0.5)

    ax[i].set_ylim(0,1)
#     ax[i].set_xlim(0,10)
    ax[i].set_xticks(np.arange(0,11,1))
    ax[i].set_xticklabels(month_names,fontsize=12)#, rotation=45)
    ax[i].grid(axis='both')
    ax[i].set_title(var, fontsize=14)
    mn = np.round(np.mean(forecast_validation['validation_stats'][var]['corr_tot']**2),2)
    ax[i].text(0.1,0.05, ("Mean R$^2$ = "+str(mn)),fontsize=14)

if Insamp==True:     
#     plt.suptitle(('LIM trained on all months \n Training years: '+str(850)+'-'+str(int(850+ntrain*1000))+
# #                  ', Validation years: Historical'+str(850)+'-'+str(int(850+ntrain*1000))), fontsize=16)
    plt.suptitle('None')
else:     
    plt.suptitle(('LIM trained on all months \n Training years: '+tcmip+' '+tmod+' '+texp+' '+
                  exp_setup['mod_filename'][-23:-14]+', Validation years: '+vmod+' '+vexp+' '+
                  valid_yrs_str), fontsize=16)

# ax[i].text(0.1,0.05, ("Training years: "+str(850)+'-'+str(850+ntrain*1000)),fontsize=14)
# ax[i].text(0.1,0.05, ("Validation years: "+str(850+ntrain*1000)+'-1850'),fontsize=14)


ax[1].legend(loc='lower right', fontsize=14)
ax[2].set_xlabel('Forecasted month', fontsize=14)
ax[4].set_xlabel('Forecasted month', fontsize=14)
ax[3].set_xlabel('Forecasted month', fontsize=14)

plt.subplots_adjust(top=0.9)

In [ ]:
fig,axs = plt.subplots(2,3,figsize=(20,10))
#fig,axs= plt.subplots(2, 2)

ax = axs.flatten()

for i,var in enumerate(limvars):
    ax[i].plot(forecast_validation['validation_stats'][var]['gm_var_ratio'],linewidth=3.0, color='purple')

    ax[i].plot(ar1cast['validation_stats_ar1'][var]['gm_var_ratio'],linewidth=3.0, color='purple', 
               linestyle='--', alpha=0.5)

#    ax[i].set_ylim(0,1)
    ax[i].set_xlim(0,10)
    ax[i].set_ylim(0,1.1)
    ax[i].set_xticks(np.arange(0,11,1))
    ax[i].set_xticklabels(month_names,fontsize=12)
    ax[i].grid(axis='both')
    ax[i].set_title(var, fontsize=16)
    ax[i].text(0.15,0.05, ("Mean ratio = "+str(np.round(np.mean(forecast_validation['validation_stats'][var]['gm_var_ratio']),2))),
               fontsize=14)
    ax[i].axhline(1.0,color='k',linestyle='--')

if Insamp==True:     
#     plt.suptitle(('LIM trained on all months \n Training years: '+str(850)+'-'+str(int(850+ntrain*1000))+
#                  ', Validation years: Historical'+str(850)+'-'+str(int(850+ntrain*1000))), fontsize=16)
    plt.suptitle(('LIM trained on all months \n Training years: '+str(850)+'-'+str(int(850+ntrain*1000))+
                 ', Validation years: Historical'), fontsize=16)
else:     
#     plt.suptitle(('LIM trained on all months \n Training years: '+str(850)+'-'+
#                   str(int(850+ntrain*1000))+', Validation years: '+
#                   str(int(850+ntrain*1000))+'-'+str(int(850+ntrain*1000+nvalidtimes/12))), fontsize=16)
    plt.suptitle(('LIM trained on all months \n Training years: '+tcmip+' '+tmod+' '+texp+' '+
                  exp_setup['mod_filename'][-23:-14]+', Validation years: '+vmod+' '+vexp+' '+
                  valid_yrs_str), fontsize=16)
    
ax[2].set_xlabel('Forecasted month', fontsize=14)
ax[1].set_xlabel('Forecasted month', fontsize=14)
ax[3].set_xlabel('Forecasted month', fontsize=14)
ax[0].set_ylabel('GM ratio: \nerror variance to true variance', fontsize=14)

plt.subplots_adjust(top=0.9)
#plt.tight_layout()